#### Which burrito trucks to select?

#### 0. Importing libraries

In [2]:
%pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
%pip install numpy
import numpy as np

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


#### 1. Set up the model

In [3]:
burrito_model = gp.Model()

Restricted license - for non-production use only - expires 2023-10-25


#### 2. Define sets and parameters

In [4]:
I = [1, 2] # list of customers
J = [1, 2, 3] # list of potential truck locations

d = [50, 40] # demand
c = np.array([[.1, 3, 5],
              [5, 3, .1]]) # distance from each customer to each potential truck location
f = 250 # setup cost per truck
r = 10 # unit burrito revenue
k = 5 # unit burrito ingredient cost

#### 3. Set the variables and the objective function

In [5]:
x = burrito_model.addVars(J, vtype = GRB.BINARY, name = "x")
y = burrito_model.addVars(I, J, vtype = GRB.BINARY, name = "y")

In [6]:
y

{(1, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 3): <gurobi.Var *Awaiting Model Update*>,
 (2, 1): <gurobi.Var *Awaiting Model Update*>,
 (2, 2): <gurobi.Var *Awaiting Model Update*>,
 (2, 3): <gurobi.Var *Awaiting Model Update*>}

In [7]:
burrito_model.setObjective(gp.quicksum(gp.quicksum((r - k) * 1/(c[i - 1, j - 1]) * d[i - 1] * y[i, j] for j in J) for i in I) - gp.quicksum(f * x[j] for j in J))

#### 4. Set the "sense" of the optimization problem

In [8]:
burrito_model.ModelSense = GRB.MAXIMIZE

#### 5. Add the constraints

In [9]:
for i in I:
    burrito_model.addConstr(gp.quicksum(y[i, j] for j in J) <= 1)

for i in I:
    for j in J:
        burrito_model.addConstr(y[i, j] <= x[j])

#### 6. Update and optimize the model

In [10]:
burrito_model.update()
burrito_model.write('burrito_truck_location.lp')
burrito_model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 8 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xd4cc80f6
Variable types: 0 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 8 rows and 9 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 4000 -0 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+03, best bound 4.000000000000e+03, gap 0.0000%


#### 7. Print the results

In [11]:
print("The optimal objective value for this problem is {burrito_model.objVal}") # optimal objective value

print("The optimal values of the decision variables are as follows:")
for var in burrito_model.getVars():
    print(f"{var.varName} = {var.x}")

The optimal objective value for this problem is {burrito_model.objVal}
The optimal values of the decision variables are as follows:
x[1] = 1.0
x[2] = 0.0
x[3] = 1.0
y[1,1] = 1.0
y[1,2] = 0.0
y[1,3] = 0.0
y[2,1] = 0.0
y[2,2] = 0.0
y[2,3] = 1.0
